In [2]:
import cv2 as cv
from glob import glob
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [4]:
def preprocess(img):
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    img_green = img[:, :, 1]
    img = np.zeros(img.shape)
    img[:, :, 1] = img_green
    img = cv.cvtColor(img.astype(np.uint8), cv.COLOR_BGR2GRAY)
    img = cv.GaussianBlur(img, (3,3), 0)
    plt.figure()
    plt.imshow(img, 'gray')
    return img

In [5]:
def prepare_tile(img, x, y, size):
    tile = img[y:y+size, x:x+size]
    return tile    

In [9]:
df = pd.DataFrame(columns=['color_average', 'color_mean', 'm1', 'm2', 'm3', 'h1', 'h2', 'h3', 'result'])
print(df)

Empty DataFrame
Columns: [color_average, color_mean, m1, m2, m3, h1, h2, h3, result]
Index: []


In [ ]:
#generating train data
files = glob('images/*')
masks = glob('manual1/*')
train_photo_number = 5
tile_size = 5
i=4
for file, mask in zip(files, masks):
    img = cv.imread(file)
    mask = cv.imread(mask)
    img = preprocess(img)
    y_size, x_size = img.shape
    
    for x in range(0, x_size-tile_size, 3):
        for y in range(0,y_size-tile_size):
            tile = prepare_tile(img, x, y, tile_size)
            #print(cv.moments(tile))
            
    i+=1
    if(i >= train_photo_number):
        break
    #for x in range(